# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

<br>
$
p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T \\
A = ((1-\alpha)M + \frac{\alpha}{N}E) \\
Opakujte: \\
\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}
$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [32]:
import numpy as np
import csv
import spacy
import re
import functools
from multiprocessing import Pool

In [5]:
def create_graph_matrix(file_path: str) -> np.array:
    graph_dict = {}
    
    with open(file_path, 'rt') as file:
        max_num = -1
        
        for line in file:
            source, dest = [int(num.strip()) - 1 for num in line.split(',')]

            if source in graph_dict:
                graph_dict[source].append(dest)
            else:
                graph_dict[source] = [dest]
                
            max_num = sorted([source, dest, max_num], reverse=True)[0]
        
    matrix = np.zeros((max_num + 1, max_num + 1))
        
    for key in graph_dict:
        
        if graph_dict[key]:
            matrix[key][graph_dict[key]] = 1 / len(graph_dict[key])
        
    return matrix


alpha = 0.2

for i in range(1, 5):
    M = create_graph_matrix(f'graph_{i}.txt')
    n = M.shape[0]
    p = np.ones((n, 1)) / n

    mask = np.sum(M, axis=1) == 0

    A = (1 - alpha) * M + (alpha / n)
    A[mask] = 1 / n

    for _ in range(100):
        p = A.transpose() @ p
    
    print(p)

[[0.0656044 ]
 [0.11808792]
 [0.16007474]
 [0.19366419]
 [0.22053575]
 [0.242033  ]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
[[0.17857143]
 [0.32142857]
 [0.32142857]
 [0.17857143]]
[[0.27257372]
 [0.15666713]
 [0.13837881]
 [0.10924643]
 [0.18531604]
 [0.06563464]
 [0.07218322]]


### Předpokládaný výstup

#### graph_1.xt
```python
array([[0.0656044 ],
       [0.11808792],
       [0.16007474],
       [0.19366419],
       [0.22053575],
       [0.242033  ]])
```
       
#### graph_2.txt
```python
array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])
```


#### graph_3.txt
```python
array([[0.17857143],
       [0.32142857],
       [0.32142857],
       [0.17857143]])
```


#### graph_4.txt
```python
array([[0.27257372],
       [0.15666713],
       [0.13837881],
       [0.10924643],
       [0.18531604],
       [0.06563464],
       [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [9]:
for i in range(3, 4):
    M = create_graph_matrix(f'graph_{i}.txt')
    M[M != 0] = 1
    n = M.shape[0]
    a = np.ones((n, 1))
    h = np.ones((n, 1))

    for _ in range(100):
        h = M @ a
        a = M.transpose() @ h
        
        h = h / np.linalg.norm(h, ord=1)
        a = a / np.linalg.norm(a, ord=1)
        
    
    print(a)
    print()
    print(h)
    print('\n')

[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]

[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]




### Předpokládaný výstup

#### graph_1.xt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

In [3]:
def load_articles(file_path: str) -> tuple:
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        data = [article for article in reader]
        return data[0], data[1:]

header, articles = load_articles('articles.csv')

In [38]:
titles = [article[4] for article in articles]


def adjust_text(text: str, to_be_removed: re.Pattern, lemmatizer) -> list:
    text = text.lower()
    text = re.sub(to_be_removed, '', text)
    text = re.sub('\s{1,}', ' ', text)
    
    return [token.lemma_ for token in lemmatizer(text)]
    

def inverted_index(title: tuple) -> list:
    index = {}
    text_adjusted = adjust_text(
        title[1],
        r'[.,?!/\\\"`\-:()\[\]*|—’–]',
        spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    )
    
    for word in text_adjusted:
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
            
    for key in index:
        index[key] = [(f'D{title[0]}', index[key])]
    
    return index

def join_indexes(index_a: dict, index_b: dict) -> dict:
    new_index = {}
    
    new_index.update(index_a)
    
    for key in index_b:
        if key in new_index:
            new_index[key].extend(index_b[key])
        else:
            new_index[key] = [index_b[key]]

    return new_index

titles_with_ids = [(i, titles[i]) for i in range(len(titles))]

with Pool(8) as p:
    sub_indexes = p.map(inverted_index, titles_with_ids)
    final_index = functools.reduce(join_indexes, sub_indexes)

print(final_index['be'])

[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]


### Předpokládaný výstup

```python
print(inverted_index['be'])
print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.